In [2]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from os import listdir
from os.path import join, isfile

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Create Training Data

In [12]:
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

def face_detector(img):
    """
    Detects faces from the image and returns a cropped face.
    If there are no faces detected, it returns the same input image.
    """
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3,5)
    if faces is ():
        return None
    #Crop the faces
    for (x,y,w,h) in faces:
        crop = img[y:y+h,x:x+w]
    
    return crop

#Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

#Collect 150 samples of your face, for training. 
while True:
        _, frame = cap.read()
        if face_detector(frame) is not None:
            count += 1
            img = cv2.resize(face_detector(frame),(200,200))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            #Save the image
            path = "./images/"+ str(count) + ".jpg"
            cv2.imwrite(path,img)
            
            #Display live count of number of samples
            cv2.putText(img, str(count), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)
            cv2.imshow("Sample", img)
            
        else:
            print("Face not found")
            break
            
        if cv2.waitKey(1) == 13 or count == 150:
            break
            
cap.release()
cv2.destroyAllWindows()
print("Sample collection complete")

Sample collection complete


### Train the model

In [13]:
file_path = "./images/"
files = [file for file in listdir(file_path) if isfile(join(file_path,file))]

#create numpy array of training data
train = []
labels = []

for idx, img in enumerate(files):
    img_path = file_path+img
    image = cv2.imread(img_path, 0)
    train.append(np.asarray(image, dtype= np.uint8))
    labels.append(idx)
    
labels = np.asarray(labels, dtype=np.int32)

#create the model and train 
model = cv2.face.LBPHFaceRecognizer_create()
model.train(np.asarray(train),labels)

print("Model trained!")

Model trained!


### Classifying Images

In [19]:
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

def face_detector(img):
    """
    Detects faces from the image and returns a cropped face.
    If there are no faces detected, it returns the same input image.
    """
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3,5)
    if faces is ():
        return img,[]
    #Crop the faces
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(240,200,150),2)
        roi = img[y:y+h,x:x+w]
        cv2.resize(roi, (200,200))
    
    return img,roi

#Initialize Webcam
cap = cv2.VideoCapture(0)

#Collect 150 samples of your face, for training. 
while True:
        _, frame = cap.read()
        img,face = face_detector(frame)
        try:
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            
            #pass the new face to model
            res = model.predict(face)
            #res is tuple of label and conf value
            
            if res[1] < 500:
                conf = int(100*(1-(res[1])/300))
                msg = str(conf)+"% confident this is the user"
            cv2.putText(img, msg, (100,120), cv2.FONT_HERSHEY_COMPLEX, 1, (150,250,255), 2)
            
            if conf > 75:
                cv2.putText(img, "UNLOCKED", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                cv2.imshow("Face", img)
                
            else:
                cv2.putText(img, "LOCKED", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
                cv2.imshow("Face", img)   
        except:
            cv2.putText(img,"No Face Found" , (220,120), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.putText(img, "LOCKED", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow("Face", img)
            pass
            
        if cv2.waitKey(1) == 13:
            break
            
cap.release()
cv2.destroyAllWindows()